In [23]:
import pandas as pd

```
DAEMON_NAME="osmosisd"
RANGE_START=$1
RANGE_END=$(( RANGE_START + 10 ))
for (( b=$RANGE_START; b<$RANGE_END; b++ )); do echo $b
        BLOCK="$(${DAEMON_NAME} q block --node https://rpc.osmosis.interbloc.org:443 $b)"
        NUM_TXS="$(echo $BLOCK | jq -r '.block.data.txs | length')"
        if [[ $NUM_TXS > 0 ]]; then
                TX_HASHES="$(for i in $(echo "${BLOCK}" | jq -r .block.data.txs[]); do echo -n $i | base64 -d | sha256sum -b - | cut -d ' ' -f 1; done)"
                for TXN in $TX_HASHES; do
                        if [ ! -e "/app/data/tx/$TXN.json" ]; then
                                ${DAEMON_NAME} q tx $TXN --output json --node https://rpc.osmosis.interbloc.org:443 > /app/data/tx/$TXN.json
                        fi
                        MSGS="$(jq -r '.tx | select(.["@type"]=="/cosmos.tx.v1beta1.Tx") | .body' "/app/data/tx/$TXN.json")"
                        NUM_MSGS="$(echo $MSGS | jq -r '.messages | length')"
                        if [[ $NUM_MSGS > 0 ]]; then
                                for (( m=0; m<$NUM_MSGS; m++ )); do
                                        echo $MSGS | jq -r " .messages[$m] | select( .[\"@type\"]==\"/osmosis.gamm.v1beta1.MsgJoinPool\" ) | [ $b, .sender, .poolId, .shareOutAmount, .tokenInMaxs[0].amount, .tokenInMaxs[0].denom, .tokenInMaxs[1].amount, .tokenInMaxs[1].denom ] | @csv" | tee -a nitrogen_joins.csv
                                done
                        fi
                done
        fi
done
```

In [24]:
def parse_msg(msg, height):

    m = {}

    if msg["@type"] == MSG_EXIT_POOL:
        m["Type"] = "Exit"
        m["Height"] = height
        m["Sender"] = msg["sender"]
        m["PoolId"] = msg["poolId"]
        m["Shares"] = msg["shareInAmount"]
        m["token[0].amount"] = msg["tokenOutMins"][0]["amount"]
        m["token[0].denom"] = msg["tokenOutMins"][0]["denom"]
        m["token[1].amount"] = msg["tokenOutMins"][1]["amount"]
        m["token[1].denom"] = msg["tokenOutMins"][1]["denom"]
    
    elif msg["@type"] == MSG_JOIN_POOL:
        m["Type"] = "Join"
        m["Height"] = height
        m["Sender"] = msg["sender"]
        m["PoolId"] = msg["poolId"]
        m["Shares"] = msg["shareOutAmount"]
        m["token[0].amount"] = msg["tokenInMaxs"][0]["amount"]
        m["token[0].denom"] = msg["tokenInMaxs"][0]["denom"]
        m["token[1].amount"] = msg["tokenInMaxs"][1]["amount"]
        m["token[1].denom"] = msg["tokenInMaxs"][1]["denom"]

    return m

def parse_tx(tx, height):
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = executor.map(parse_msg, tx["body"]["messages"], repeat(height))
        for m in results:
            if m:
                print(','.join(str(x) for x in m.values()))

def call_endpoint(url, headers = {'Accept': 'application/json'}):
    response = requests.get(url, headers=headers)
    return response.json()

In [40]:
import requests
import os
import pandas as pd
from itertools import repeat
import concurrent.futures
import base64
import hashlib

UPGRADE_HEIGHT=4707300
HALT_HEIGHT=4713064

# API_ENDPOINT="https://api.osmosis.interbloc.org/cosmos/tx/v1beta1/txs?events=tx.height={}"
RPC_ENDPOINT="https://rpc.osmosis.interbloc.org/"

MSG_EXIT_POOL="/osmosis.gamm.v1beta1.MsgExitPool"
MSG_JOIN_POOL="/osmosis.gamm.v1beta1.MsgJoinPool"

START_HEIGHT = int(os.getenv("START_HEIGHT", UPGRADE_HEIGHT))
END_HEIGHT = int(os.getenv("END_HEIGHT", HALT_HEIGHT))

results = []

for h in range(START_HEIGHT, END_HEIGHT):
    url = RPC_ENDPOINT + "/block?height=" + str(h)
    response_json = call_endpoint(url)
    
    raw_txs = response_json["result"]["block"]["data"]["txs"]

    for tx in raw_txs:
        tx = base64.b64decode(tx)
        if MSG_EXIT_POOL in str(tx) or MSG_JOIN_POOL in str(tx):
            tx_hash = hashlib.sha256(tx).hexdigest()
            results.append((h, tx_hash))
    
df = pd.DataFrame(results, columns=["height", "tx_hash"])
df.to_csv("data/0_tx_hash.csv", index=None)


In [37]:
display(df.head())
df.describe()

,height,tx_hash
0,4713062,098895f05128b63e44e52d6f5a70c04527bf8d4a409754...
1,4713062,412e31bf4dafd475dd42e253209ac2a9566e979ed08de1...
2,4713062,380a10103f53296a1ce663fda5cf79da904e0ec175c866...
3,4713062,4f7f4df4568e5441cd88bb0bef8b7c5eb6706cef26e1cf...
4,4713062,54fbda38b3e82cac7d7de018aeb43a8a335c97f03d70df...


,height
count,1.000000e+01
mean,4.713062e+06
std,4.830459e-01
min,4.713062e+06
25%,4.713062e+06
50%,4.713062e+06
75%,4.713063e+06
max,4.713063e+06
